Let's make a dummy dataframe as data to work with. We take 30 random dates in the year 2024-2025. And assign random demand quantities to be fulfilled on these dates. One dataframe corresponds to one unique product only.

In [45]:
import pandas as pd
import random
from datetime import datetime

# Generate the 1st of every month in 2024
dates_2024 = [datetime(2024, month, 1).strftime("%d/%m/%Y") for month in range(1, 13)]

# Generate random demands for each date
random_demands = [random.randint(50, 1000) for _ in dates_2024]

# Create a DataFrame with the generated dates and demands
product_demand_df = pd.DataFrame({
    "Date": dates_2024,
    "Demand": random_demands
})

print(product_demand_df)

          Date  Demand
0   01/01/2024     119
1   01/02/2024      70
2   01/03/2024     688
3   01/04/2024     346
4   01/05/2024     918
5   01/06/2024     110
6   01/07/2024     508
7   01/08/2024     941
8   01/09/2024     598
9   01/10/2024     571
10  01/11/2024     879
11  01/12/2024     105


Now, we implement the dynamic lot-sizing problem using the Wagner Whitin Algorithm which seeks to minimise costs for a period while meeting the demand for every period. 

In [47]:
import pandas as pd
import sys

product_demand_df["Date"] = pd.to_datetime(product_demand_df["Date"])

# Extract demand values
d = product_demand_df["Demand"].tolist()
a = [10000] * len(d)  # Setup cost for each period
h = [1] * len(d)  # Holding cost for each period

def wagner_whitin(d, h, a):
    number_of_weeks = len(d)
    z_stars = []  # Optimal cost values for each period
    j_stars = []  # Associated week indices for the optimal cost

    # Step 1 to number of weeks
    for i in range(0, number_of_weeks):
        production_options = []  # Cost options for fulfilling demand up to week i
        print(f"---- PLANNED HORIZON {i + 1} ----")
        for j in range(0, (i + 1)):
            current_cost = 0
            if j != 0:  # Add previous week's optimal cost
                current_cost += z_stars[j - 1]
            current_cost += a[j]  # Add setup cost

            # Add holding cost for demands fulfilled in advance
            for iteration in range(j, i + 1):
                holding_cost = sum(h[j:iteration]) * d[iteration]
                current_cost += holding_cost

            production_options.append(current_cost)
            print(f"Last week {j + 1}: {current_cost}")

        # Select the minimum cost for the current horizon
        z_star = min(production_options)
        z_stars.append(z_star)
        max_week = production_options.index(z_star)
        j_stars.append(max_week)

    return {"z_stars": z_stars, "j_stars": j_stars}

answers = wagner_whitin(d, h, a)
z_stars = answers["z_stars"]
j_stars = answers["j_stars"]

# Output optimal costs and associated weeks
print("---Z values---")
for i in range(len(z_stars)):
    print(f"z_{i + 1}: {z_stars[i]}")

print("---J values---")
for i in range(len(j_stars)):
    print(f"j_{i + 1}: {j_stars[i] + 1}")

# Determine production lot sizes and schedule
start_week = len(j_stars) - 1
end_week = len(j_stars) - 1
lot_number = 1
lot_size = 0

while True:
    start_week = j_stars[end_week]
    print(f"---Lot number: {lot_number}---")
    print(f"Start week: {start_week + 1}")
    print(f"End week: {end_week + 1}")
    for i in range(start_week, end_week + 1):
        lot_size += d[i]
    print(f"Lot size: {lot_size}")

    if start_week == 0:
        break

    lot_size = 0
    lot_number += 1
    end_week = start_week - 1

---- PLANNED HORIZON 1 ----
Last week 1: 10000
---- PLANNED HORIZON 2 ----
Last week 1: 10070
Last week 2: 20000
---- PLANNED HORIZON 3 ----
Last week 1: 11446
Last week 2: 20688
Last week 3: 20070
---- PLANNED HORIZON 4 ----
Last week 1: 12484
Last week 2: 21380
Last week 3: 20416
Last week 4: 21446
---- PLANNED HORIZON 5 ----
Last week 1: 16156
Last week 2: 24134
Last week 3: 22252
Last week 4: 22364
Last week 5: 22484
---- PLANNED HORIZON 6 ----
Last week 1: 16706
Last week 2: 24574
Last week 3: 22582
Last week 4: 22584
Last week 5: 22594
Last week 6: 26156
---- PLANNED HORIZON 7 ----
Last week 1: 19754
Last week 2: 27114
Last week 3: 24614
Last week 4: 24108
Last week 5: 23610
Last week 6: 26664
Last week 7: 26706
---- PLANNED HORIZON 8 ----
Last week 1: 26341
Last week 2: 32760
Last week 3: 29319
Last week 4: 27872
Last week 5: 26433
Last week 6: 28546
Last week 7: 27647
Last week 8: 29754
---- PLANNED HORIZON 9 ----
Last week 1: 31125
Last week 2: 36946
Last week 3: 32907
Last we

Final Results: 

---Lot number: 1---
Start week: 8
End week: 12
Lot size: 3094
---Lot number: 2---
Start week: 1
End week: 7
Lot size: 2759


Now, let us try and introduce a warehouse capacity into the problem.

In [48]:
import pandas as pd
import sys

# Add warehouse capacity
warehouse_capacity = 1000

# Ensure product_demand_df exists (use your generated demand DataFrame)
product_demand_df["Date"] = pd.to_datetime(product_demand_df["Date"])

# Extract demand values
d = product_demand_df["Demand"].tolist()
a = [10000] * len(d)  # Setup cost for each period
h = [1] * len(d)  # Holding cost for each period

def wagner_whitin_with_capacity(d, h, a, capacity):
    number_of_weeks = len(d)
    z_stars = []  # Optimal cost values for each period
    j_stars = []  # Associated week indices for the optimal cost

    for i in range(0, number_of_weeks):
        production_options = []  # Cost options for fulfilling demand up to week i
        print(f"---- PLANNED HORIZON {i + 1} ----")
        for j in range(0, (i + 1)):
            current_cost = 0
            total_demand = sum(d[j:i + 1])  # Calculate total demand from week j to i

            # Check warehouse capacity constraint
            if total_demand > capacity:
                production_options.append(float("inf"))  # Infeasible option
                print(f"Last week {j + 1}: Infeasible (exceeds capacity)")
                continue

            if j != 0:  # Add previous week's optimal cost
                current_cost += z_stars[j - 1]
            current_cost += a[j]  # Add setup cost

            # Add holding cost for demands fulfilled in advance
            for iteration in range(j, i + 1):
                holding_cost = sum(h[j:iteration]) * d[iteration]
                current_cost += holding_cost

            production_options.append(current_cost)
            print(f"Last week {j + 1}: {current_cost}")

        # Select the minimum cost for the current horizon
        z_star = min(production_options)
        z_stars.append(z_star)
        max_week = production_options.index(z_star)
        j_stars.append(max_week)

    return {"z_stars": z_stars, "j_stars": j_stars}

answers = wagner_whitin_with_capacity(d, h, a, warehouse_capacity)
z_stars = answers["z_stars"]
j_stars = answers["j_stars"]

# Output optimal costs and associated weeks
print("---Z values---")
for i in range(len(z_stars)):
    print(f"z_{i + 1}: {z_stars[i]}")

print("---J values---")
for i in range(len(j_stars)):
    print(f"j_{i + 1}: {j_stars[i] + 1}")

# Determine production lot sizes and schedule
start_week = len(j_stars) - 1
end_week = len(j_stars) - 1
lot_number = 1
lot_size = 0

while True:
    start_week = j_stars[end_week]
    print(f"---Lot number: {lot_number}---")
    print(f"Start week: {start_week + 1}")
    print(f"End week: {end_week + 1}")
    for i in range(start_week, end_week + 1):
        lot_size += d[i]
    print(f"Lot size: {lot_size}")

    if lot_size > warehouse_capacity:
        print(f"Warning: Lot size {lot_size} exceeds capacity!")

    if start_week == 0:
        break

    lot_size = 0
    lot_number += 1
    end_week = start_week - 1

---- PLANNED HORIZON 1 ----
Last week 1: 10000
---- PLANNED HORIZON 2 ----
Last week 1: 10070
Last week 2: 20000
---- PLANNED HORIZON 3 ----
Last week 1: 11446
Last week 2: 20688
Last week 3: 20070
---- PLANNED HORIZON 4 ----
Last week 1: Infeasible (exceeds capacity)
Last week 2: Infeasible (exceeds capacity)
Last week 3: Infeasible (exceeds capacity)
Last week 4: 21446
---- PLANNED HORIZON 5 ----
Last week 1: Infeasible (exceeds capacity)
Last week 2: Infeasible (exceeds capacity)
Last week 3: Infeasible (exceeds capacity)
Last week 4: Infeasible (exceeds capacity)
Last week 5: 31446
---- PLANNED HORIZON 6 ----
Last week 1: Infeasible (exceeds capacity)
Last week 2: Infeasible (exceeds capacity)
Last week 3: Infeasible (exceeds capacity)
Last week 4: Infeasible (exceeds capacity)
Last week 5: Infeasible (exceeds capacity)
Last week 6: 41446
---- PLANNED HORIZON 7 ----
Last week 1: Infeasible (exceeds capacity)
Last week 2: Infeasible (exceeds capacity)
Last week 3: Infeasible (exceed

Therefore, it works when the warehouse capacity is limited too.